In [1]:
from pathlib import Path
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nltk.tokenize import WordPunctTokenizer

# Data base import

In [2]:
train_data = pd.read_csv(Path("../data/samples/sample_1000_train.csv")).to_numpy()
validation_data = pd.read_csv(Path("../data/samples/sample_100_validation.csv")).to_numpy()

In [3]:
print(train_data.shape, validation_data.shape)

(1000, 4) (100, 4)


In [4]:
from utils.clean_data import clean_data 

train_data = clean_data(train_data)
traivalidation_data = clean_data(validation_data)

In [5]:
print(train_data.shape, validation_data.shape)

(966, 4) (100, 4)


# Pre-processing

In [6]:
# -- Get the original tweets -- #

X_TR_ORIGINAL = train_data[:, 1]
X_VAL_ORIGINAL = validation_data[:, 1]

Y_TR_ORIGINAL = train_data[:, 2]
Y_VAL_ORIGINAL = validation_data[:, 2]

FEELINGS_TR = train_data[:, 3]
FEELINGS_VAL = validation_data[:, 3]

In [7]:
from descriptors.tokenizer.tokenizer import Tokenizer
from descriptors.tweet_string.create_strings import create_strings

# -- Define the tokenizer -- #
TOKENIZER = Tokenizer()

# -- Create sentences -- #
X_TR_STRING = create_strings(X_TR_ORIGINAL, TOKENIZER)
X_VAL_STRING = create_strings(X_VAL_ORIGINAL, TOKENIZER)

Y_TR_STRING = create_strings(X_TR_ORIGINAL, TOKENIZER)
Y_VAL_STRING = create_strings(X_VAL_ORIGINAL, TOKENIZER)

print(X_VAL_STRING[0])

['Looking', 'forward', 'to', 'a', 'short', 'work', 'week', 'followed', 'by', 'a', 'mini-vacation', 'in', 'Clemson']


In [8]:
from descriptors.descriptor_ascii.load_data import load_data
from descriptors.tweet_label.create_labels import create_labels

# ALPHANUM_ONLY = False
# SPLIT_PUNCTUATION = False  # to tell wheter the puncutation "!?.;,/" etc are keeped sticked to a word or not

WORD_SIZE = 30
SENTENCE_SIZE = 50
FILL_WITH = "$"
FEELING_WEIGHT = 0


# training set --------------------------------------------
X_TR_STRING_FILLED, X_TR_SCALAR = load_data(
    X_TR_STRING, WORD_SIZE, SENTENCE_SIZE, FILL_WITH, FEELING_WEIGHT, feelings=FEELINGS_TR)

Y_TR = create_labels(X_TR_STRING, Y_TR_STRING, SENTENCE_SIZE)


# validation set ------------------------------------------
X_VAL_STRING, X_VAL_SCALAR = load_data(
    X_VAL_STRING, WORD_SIZE, SENTENCE_SIZE, FILL_WITH, FEELING_WEIGHT, feelings=FEELINGS_VAL)

Y_VAL = create_labels(X_VAL_STRING, Y_VAL_STRING, SENTENCE_SIZE)

In [16]:
IDX = 0
print("Original data :")
print(X_TR_ORIGINAL[IDX])
print("\nFilled sentence :")
print(X_TR_STRING[IDX])
print("\nLabel :")
print(Y_TR[IDX])
print("\nDescriptor :")
print(X_TR_SCALAR[IDX])

Original data :
i feel really weird

Filled sentence :
['i', 'feel', 'really', 'weird']

Label :
[1. 1. 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0.]

Descriptor :
[105  36  36 ...  36  36   0]


### Standardisation

In [17]:
mu = np.concatenate([X_TR_SCALAR[:, :-1].mean(axis=0), [0]])
sigma = X_TR_SCALAR[:, :-1].std(axis=0)

X_TR_SCALAR = X_TR_SCALAR - mu
X_VAL_SCALAR = X_VAL_SCALAR - mu

for i in range(len(sigma)):
    if sigma[i] != 0:
        X_TR_SCALAR[:, i] /= sigma[i]
        X_VAL_SCALAR[:, i] /= sigma[i]

# Classification

In [9]:
#!pip install xgboost

In [18]:
from sklearn.multioutput import MultiOutputRegressor

from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
import xgboost as xgb

In [19]:
nb_neighbors = 10
regressor = KNeighborsRegressor(nb_neighbors, weights="distance", metric="jaccard")
# regressor = KNeighborsRegressor(nb_neighbors, weights="distance")

# regressor = MultiOutputRegressor(
#     xgb.XGBClassifier(objective="reg:logistic")
# )

#regressor = MultiOutputRegressor(
#    SVR(kernel="poly", degree=2)
#)

regressor.fit(X_TR_SCALAR, Y_TR)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='jaccard',
                    metric_params=None, n_jobs=None, n_neighbors=10, p=2,
                    weights='distance')

In [20]:
predictions = regressor.predict(X_VAL_SCALAR)

In [21]:
from utils.post_processing import preds_to_strings

results_val = preds_to_strings(X_VAL_ORIGINAL, X_VAL_STRING, predictions)

In [22]:
from utils.loss import mean_jaccard

accuracy = mean_jaccard(validation_data[:, 2], results_val)
print(accuracy)

54.38447342142759


In [23]:
def test_param(n):
    regressor = KNeighborsRegressor(n, weights="distance", metric="jaccard")
    regressor.fit(X_TR_SCALAR, Y_TR)
    predictions = regressor.predict(X_VAL_SCALAR)
    results_val = preds_to_strings(X_VAL_ORIGINAL, X_VAL_STRING, predictions)
    accuracy = mean_jaccard(validation_data[:, 2], results_val)
    
    return accuracy

In [24]:
y =[]
for i in range(2,50, 3):
    y.append(test_param(i))
plt.plot(y)
plt.show()

KeyboardInterrupt: 